<a href="https://colab.research.google.com/github/SiracencoSerghei/DataScienceHW/blob/main/example_kaggle/les_7/les_7_fashion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install scikit-surprise

In [22]:
import pandas as pd
import os

from surprise import Dataset, Reader, SVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [23]:
# # Set the path to the base folder containing the "example_kaggle/les_7/fashion_products" folder.
# # Adjust the path as needed.
# path = '/home/sergio/Desktop/DataScienceHW/example_kaggle/les_7'

# # Build paths to the MovieLens CSV files
# path = os.path.join(path, 'fashion_products')
# fashion_file = os.path.join(path, 'fashion_products.csv')
# fashion_file


In [24]:
from google.colab import files
files.upload()

Saving fashion_products.csv to fashion_products (2).csv


{'fashion_products (2).csv': b"User ID,Product ID,Product Name,Brand,Category,Price,Rating,Color,Size\r\n19,1,Dress,Adidas,Men's Fashion,40,1.0431592108361825,Black,XL\r\n97,2,Shoes,H&M,Women's Fashion,82,4.026416271141911,Black,L\r\n25,3,Dress,Adidas,Women's Fashion,44,3.337937559377053,Yellow,XL\r\n57,4,Shoes,Zara,Men's Fashion,23,1.0495229563128543,White,S\r\n79,5,T-shirt,Adidas,Men's Fashion,79,4.302773408398684,Black,M\r\n98,6,Dress,Adidas,Men's Fashion,47,1.3795657395330458,Yellow,L\r\n16,7,Jeans,Gucci,Men's Fashion,37,1.3567503746842564,White,XL\r\n63,8,Sweater,Zara,Kids' Fashion,64,4.36030328941572,Blue,XL\r\n96,9,Sweater,H&M,Men's Fashion,53,4.466181876278437,Green,XL\r\n36,10,T-shirt,Zara,Kids' Fashion,55,4.093234402033421,White,XL\r\n69,11,T-shirt,Adidas,Men's Fashion,51,1.1609879573062742,Red,S\r\n87,12,Sweater,Gucci,Kids' Fashion,91,2.699735537578599,Yellow,M\r\n9,13,Jeans,Nike,Kids' Fashion,35,1.6011935800534172,Red,M\r\n50,14,Dress,Zara,Women's Fashion,34,2.9210042039473

In [34]:
# data = pd.read_csv(fashion_file)
data = pd.read_csv('fashion_products.csv')
data = pd.DataFrame(data)
print(data.head())

   User ID  Product ID Product Name   Brand         Category  Price    Rating  \
0       19           1        Dress  Adidas    Men's Fashion     40  1.043159   
1       97           2        Shoes     H&M  Women's Fashion     82  4.026416   
2       25           3        Dress  Adidas  Women's Fashion     44  3.337938   
3       57           4        Shoes    Zara    Men's Fashion     23  1.049523   
4       79           5      T-shirt  Adidas    Men's Fashion     79  4.302773   

    Color Size  
0   Black   XL  
1   Black    L  
2  Yellow   XL  
3   White    S  
4   Black    M  


---
# First Approach: Content-Based Filtering
---

In [35]:
type(data)


pandas.core.frame.DataFrame

In [36]:
content_df = data[['Product ID', 'Product Name', 'Brand',
                   'Category', 'Color', 'Size']]
content_df = content_df.copy()
# content_df['Content'] = content_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
content_df.loc[:, 'Content'] = content_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)


# Use TF-IDF vectorizer to convert content into a matrix of TF-IDF features
tfidf_vectorizer = TfidfVectorizer()
content_matrix = tfidf_vectorizer.fit_transform(content_df['Content'])

content_similarity = linear_kernel(content_matrix, content_matrix)

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(data[['User ID',
                                  'Product ID',
                                  'Rating']], reader)

def get_content_based_recommendations(product_id, top_n):
    index = content_df[content_df['Product ID'] == product_id].index[0]
    similarity_scores = content_similarity[index]
    similar_indices = similarity_scores.argsort()[::-1][1:top_n + 1]
    recommendations = content_df.loc[similar_indices, 'Product ID'].values
    return recommendations

---
# Second Approach: Collaborative Filtering
---

In [37]:
algo = SVD()
trainset = data.build_full_trainset()
algo.fit(trainset)

def get_collaborative_filtering_recommendations(user_id, top_n):
    testset = trainset.build_anti_testset()
    testset = filter(lambda x: x[0] == user_id, testset)
    predictions = algo.test(testset)
    predictions.sort(key=lambda x: x.est, reverse=True)
    recommendations = [prediction.iid for prediction in predictions[:top_n]]
    return recommendations